In [9]:
import pandas as pd
import os
from openpyxl import load_workbook
import numpy as np
import re

import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
import sklearn.feature_selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import MultiTaskElasticNetCV
from sklearn.base import clone
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV




In [10]:
#adatok importálása
ss_df=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ss_df.csv")
ss_df2=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ss_df2.csv")
mm_df=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\mm_df.csv")
mm_df2=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\mm_df2.csv")
robust_df=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\robust_df.csv")
robust_df2=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\robust_df2.csv")

ss_df_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\ss_df_ae.csv")
ss_df2_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\ss_df2_ae.csv")
mm_df2_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\mm_df2_ae.csv")
mm_df_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\mm_df_ae.csv")
robust_df2_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\robust_df2_ae.csv")
robust_df_ae=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\ae\\robust_df_ae.csv")

ss_df_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\ss_df_pca.csv")
ss_df2_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\ss_df2_pca.csv")
mm_df2_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\mm_df2_pca.csv")
mm_df_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\mm_df_pca.csv")
robust_df2_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\robust_df2_pca.csv")
robust_df_pca=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\pca\\robust_df_pca.csv")

incm=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\incm.csv")
incm2=pd.read_csv("C:\\Users\\Andris\\Documents\\Szakdoga\\scaled\\incm2.csv")
incm2.index=incm2["Unnamed: 0"]
incm2.drop(columns="Unnamed: 0", inplace=True)

incm.index=incm["Unnamed: 0"]
incm.drop(columns="Unnamed: 0", inplace=True)
mm_df

,Unnamed: 0,Malaysia: FPCPITOTLZGMYS (linear),Malaysia: FPCPITOTLZGMYS (time),Malaysia: uncertainty (linear),Malaysia: uncertainty (time),"Malaysia: Population, total","Malaysia: GNI, Atlas method (current US$)",Malaysia: GDP (current US$),"Malaysia: Inflation, GDP deflator (annual %)","Malaysia: Agriculture, forestry, and fishing, value added (% of GDP)",...,Indonesia: Linear interpolation: Adjusted net national income (current US$)_lag24,Indonesia: Linear interpolation: Adjusted net national income per capita (current US$)_lag24,Indonesia: real_gdp_ind_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_seas_adj (time)_lag24,Indonesia: real_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_non_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (time)_lag24
0,2010-03-01,0.602838,0.601282,0.258185,0.258163,0.000000,0.000000,0.000000,0.000000,0.517442,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2010-04-01,0.628647,0.626738,0.260167,0.260167,0.007973,0.012782,0.027759,0.111111,0.534884,...,0.009085,0.010496,0.005857,0.005857,0.008393,0.008393,0.015629,0.015629,0.015083,0.015083
2,2010-05-01,0.654456,0.653042,0.173444,0.172502,0.015946,0.025563,0.055519,0.222222,0.552326,...,0.018170,0.020993,0.011267,0.011326,0.016748,0.016839,0.033361,0.033553,0.030453,0.030620
3,2010-06-01,0.680265,0.678497,0.086722,0.087665,0.023919,0.038345,0.083278,0.333333,0.569767,...,0.027255,0.031489,0.016677,0.016618,0.025102,0.025012,0.051093,0.050900,0.045822,0.045655
4,2010-07-01,0.706074,0.704801,0.000000,0.000000,0.031892,0.051126,0.111038,0.444444,0.587209,...,0.036340,0.041986,0.022086,0.022086,0.033457,0.033457,0.068824,0.068824,0.061191,0.061191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2021-08-01,0.826710,0.826258,0.837269,0.835500,0.976178,0.959079,0.923443,0.622389,0.449612,...,0.967928,0.970037,0.984514,0.984599,0.990611,0.990662,0.987446,0.987310,0.994147,0.994083
138,2021-09-01,0.841707,0.841049,0.674538,0.676306,0.982133,0.969309,0.942583,0.676692,0.476744,...,0.975946,0.977528,0.992257,0.992173,0.995306,0.995255,0.974893,0.975029,0.988293,0.988357
139,2021-10-01,0.856704,0.856334,0.511806,0.511806,0.988089,0.979540,0.961722,0.730994,0.503876,...,0.983964,0.985018,1.000000,1.000000,1.000000,1.000000,0.962339,0.962339,0.982440,0.982440
140,2021-11-01,0.871700,0.871125,0.405866,0.408169,0.994044,0.989770,0.980861,0.785297,0.531008,...,0.991982,0.992509,0.995570,0.995666,0.998756,0.998783,0.945268,0.945639,0.971003,0.971252


In [11]:
#kezeletlen linreg scaled datával

def linreg(df, inc):
    y=inc['incidents_per_month']
    df=df
    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)
    x=df
    x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.2, random_state = 0) 
    
    lm = LinearRegression() 
    lm.fit(x_train, y_train) 
    pred = lm.predict(x_test) 
    #df['x_train+Predicted']=pd.concat([pd.Series(y_train), pd.Series(pred)])
    has=dict(zip(pd.Series(y_test), pd.Series(pred)))

    #print(y_test)
    mse = mean_squared_error(y_test, pred)
    print(mse)


    return y_test, pd.Series(pred)




In [12]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
incm
ssincm=StandardScaler().fit_transform(incm.to_numpy())
ssi=[]
for x in ssincm:
    ssi.append(float(x[0]))
    
mmincm=MinMaxScaler().fit_transform(incm.to_numpy())
mmi=[]
for x in mmincm:
    mmi.append(float(x[0]))

rincm=RobustScaler().fit_transform(incm.to_numpy())
ri=[]
for x in rincm:
   ri.append(float(x[0]))


ssincm2=StandardScaler().fit_transform(incm2.to_numpy())
ssi2=[]
for x in ssincm2:
    ssi2.append(float(x[0]))
    
mmincm2=MinMaxScaler().fit_transform(incm2.to_numpy())
mmi2=[]
for x in mmincm2:
    mmi2.append(float(x[0]))

rincm2=RobustScaler().fit_transform(incm2.to_numpy())
ri2=[]
for x in rincm2:
   ri2.append(float(x[0]))



In [13]:
def ldf(l):
    l=pd.DataFrame(l, columns=["incidents_per_month"])
    return l

ri=ldf(ri)
mmi=ldf(mmi)
ssi=ldf(ssi)

ri2=ldf(ri2)
mmi2=ldf(mmi2)
ssi2=ldf(ssi2)


In [14]:

ss_df_lr=linreg(ss_df, incm)
mm_df_lr=linreg(mm_df, incm)
robust_df_lr=linreg(robust_df, incm)


ss_df2_lr=linreg(ss_df2, incm2)
mm_df2_lr=linreg(mm_df2, incm2)
robust_df2_lr=linreg(robust_df2, incm2)

#legjobban teljesítő: mm_df2_pca

#linreg az adott scalinghez hasonlóan scaled incidens adatokkal


ss_df_lr=linreg(ss_df, ssi)
mm_df_lr=linreg(mm_df, mmi) #ez a legjobb so far
robust_df_lr=linreg(robust_df, ri)


ss_df2_lr=linreg(ss_df2, ssi2)
mm_df2_lr=linreg(mm_df2, mmi2)
robust_df2_lr=linreg(robust_df2, ri2)

17.634283370135385
18.04099573027588
16.619708851182683
45.71787518391064
47.13003601940499
51.64374001989177
1.694574684989541
0.04090928737024014
1.846634316797844
4.461767216970729
0.10687030291549156
3.6724437347477763


In [15]:
#linreg recursive feature eliminationnel

from sklearn.feature_selection import RFE


def rfelinreg(df, inc):
    #df.index=df['Unnamed: 0']
    #df.drop(columns= "Unnamed: 0", inplace=True)
    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)

    x = df
    y = inc['incidents_per_month']


    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    lr = LinearRegression()
    rfe = RFE(estimator=lr, n_features_to_select=int(len(df)/10))  
    rfe.fit(X_train, y_train)

    sel= x.columns[rfe.support_]

    X_trainsel = X_train[sel]
    X_testsel= X_test[sel]

    lr.fit(X_trainsel, y_train)
    y_pred = lr.predict(X_testsel)

    mse = mean_squared_error(y_test, y_pred)
    print(mse)
    return mse

#kell ez egyáltalán? ez már alapból dimenziócsökkentve van. Sztem kiszedhető
ss_df_ae_pred_rfe=rfelinreg(ss_df_ae, incm)
ss_df2_ae_pred_rfe=rfelinreg(ss_df2_ae, incm2)
mm_df_ae_pred_rfe=rfelinreg(mm_df_ae, incm)
mm_df2_ae_pred_rfe=rfelinreg(mm_df2_ae, incm2)
robust_df_ae_pred_rfe=rfelinreg(robust_df_ae, incm)
robust_df2_ae_pred_rfe=rfelinreg(robust_df2_ae, incm2)

ss_df_pca_pred_rfe=rfelinreg(ss_df_pca, incm)
ss_df2_pca_pred_rfe=rfelinreg(ss_df2_pca, incm2)
mm_df_pca_pred_rfe=rfelinreg(mm_df_pca, incm)
mm_df2_pca_pred_rfe=rfelinreg(mm_df2_pca, incm2)
robust_df_pca_pred_rfe=rfelinreg(robust_df_pca, incm)
robust_df2_pca_pred_rfe=rfelinreg(robust_df2_pca, incm2)




#legjobban teljesítő: mm_df2_pca
#ugyanez+grad descent/early stopping?

7.001061209724694
5.406642677449879
9.839192002471997
5.978243740614692
15.42432923460355
7.599755274895037
8.123401123904637
5.982637664541154
7.13150421937276
4.634656060254953
16.760696458987805
8.039614937904645


In [16]:
ss_df_pred_rfe=rfelinreg(ss_df, incm)
ss_df2_pred_rfe=rfelinreg(ss_df2, incm2)
mm_df_pred_rfe=rfelinreg(mm_df, incm)
mm_df2_pred_rfe=rfelinreg(mm_df2, incm2)
robust_df_pred_rfe=rfelinreg(robust_df, incm)
robust_df2_pred_rfe=rfelinreg(robust_df2, incm2)

#legjobban teljesítő: mm_df2

print("scaled y-nal:")
ss_df_pred_rfe=rfelinreg(ss_df, ssi)
ss_df2_pred_rfe=rfelinreg(ss_df2, ssi2)
mm_df_pred_rfe=rfelinreg(mm_df, mmi)
mm_df2_pred_rfe=rfelinreg(mm_df2, mmi2)
robust_df_pred_rfe=rfelinreg(robust_df, ri)
robust_df2_pred_rfe=rfelinreg(robust_df2, ri2)

9.700368464073275
7.126480747554354
13.382818593635397
6.88156135740987
15.42432923460355
7.599755274895037
scaled y-nal:
0.932161431755597
0.695498162237425
0.030346527423210173
0.015604447522469518
1.713814359399961
0.5404270417702979


In [17]:
mm_df

,Malaysia: FPCPITOTLZGMYS (linear),Malaysia: FPCPITOTLZGMYS (time),Malaysia: uncertainty (linear),Malaysia: uncertainty (time),"Malaysia: Population, total","Malaysia: GNI, Atlas method (current US$)",Malaysia: GDP (current US$),"Malaysia: Inflation, GDP deflator (annual %)","Malaysia: Agriculture, forestry, and fishing, value added (% of GDP)","Malaysia: Population, total.1",...,Indonesia: Linear interpolation: Adjusted net national income (current US$)_lag24,Indonesia: Linear interpolation: Adjusted net national income per capita (current US$)_lag24,Indonesia: real_gdp_ind_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_seas_adj (time)_lag24,Indonesia: real_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: real_gdp_ind_non_seas_adj (time)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (linear)_lag24,Indonesia: nominal_gdp_ind_non_seas_adj (time)_lag24
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2010-03-01,0.602838,0.601282,0.258185,0.258163,0.000000,0.000000,0.000000,0.000000,0.517442,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2010-04-01,0.628647,0.626738,0.260167,0.260167,0.007973,0.012782,0.027759,0.111111,0.534884,0.007861,...,0.009085,0.010496,0.005857,0.005857,0.008393,0.008393,0.015629,0.015629,0.015083,0.015083
2010-05-01,0.654456,0.653042,0.173444,0.172502,0.015946,0.025563,0.055519,0.222222,0.552326,0.015984,...,0.018170,0.020993,0.011267,0.011326,0.016748,0.016839,0.033361,0.033553,0.030453,0.030620
2010-06-01,0.680265,0.678497,0.086722,0.087665,0.023919,0.038345,0.083278,0.333333,0.569767,0.023845,...,0.027255,0.031489,0.016677,0.016618,0.025102,0.025012,0.051093,0.050900,0.045822,0.045655
2010-07-01,0.706074,0.704801,0.000000,0.000000,0.031892,0.051126,0.111038,0.444444,0.587209,0.031969,...,0.036340,0.041986,0.022086,0.022086,0.033457,0.033457,0.068824,0.068824,0.061191,0.061191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,0.826710,0.826258,0.837269,0.835500,0.976178,0.959079,0.923443,0.622389,0.449612,0.976120,...,0.967928,0.970037,0.984514,0.984599,0.990611,0.990662,0.987446,0.987310,0.994147,0.994083
2021-09-01,0.841707,0.841049,0.674538,0.676306,0.982133,0.969309,0.942583,0.676692,0.476744,0.981992,...,0.975946,0.977528,0.992257,0.992173,0.995306,0.995255,0.974893,0.975029,0.988293,0.988357
2021-10-01,0.856704,0.856334,0.511806,0.511806,0.988089,0.979540,0.961722,0.730994,0.503876,0.988060,...,0.983964,0.985018,1.000000,1.000000,1.000000,1.000000,0.962339,0.962339,0.982440,0.982440


In [27]:
#szűrés másképpen (korrelációs mátrix alapján), csak scaled y-nal
def corrfilter(df, inc):
    
    df=pd.DataFrame(df)
    #print(df.head())
    if "inc" not in df.columns:
        df["inc"]=inc
    #print(df["inc"])

    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)
    #print(df.head())
   

    c=df.corr()
    
    df.drop(list(c['inc'][abs(c['inc'])<0.4].index), axis=1, inplace=True)
    
    print(len(df.columns))
    return df

mm_df2_corr=corrfilter(mm_df2, mmi2)
mm_df_corr=corrfilter(mm_df, mmi)
ss_df_corr=corrfilter(ss_df, ssi)
ss_df2_corr=corrfilter(ss_df2, ssi2)
robust_df_corr=corrfilter(robust_df, ri)
robust_df2_corr=corrfilter(robust_df2, ri2)

#robust_df

557
895
895
557
895
557


In [28]:
#linreg corr alapján szűrtekkel
def linreg2(df, inc):
    y=inc['incidents_per_month']

    #df.drop(columns= "inc", inplace=True)
    x=df
    x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.2, random_state = 0) 
    
    lm = LinearRegression() 
    lm.fit(x_train, y_train) 
    pred = lm.predict(x_test) 
    #df['x_train+Predicted']=pd.concat([pd.Series(y_train), pd.Series(pred)])
    has=dict(zip(pd.Series(y_test), pd.Series(pred)))

    #print(y_test)
    mse = mean_squared_error(y_test, pred)
    print(mse)


    return y_test, pd.Series(pred)


mm_df2_corr_lr=linreg2(mm_df2_corr, mmi2)
mm_df_corr_lr=linreg2(mm_df_corr, mmi)
ss_df2_corr_lr=linreg2(ss_df2_corr, ssi2)
ss_df_corr_lr=linreg2(mm_df_corr, ssi)
robust_df2_corr_lr=linreg2(robust_df2_corr, ri2)
robust_df_corr_lr=linreg2(robust_df_corr, ri)




ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
#linreg+early stopping
#!!!!!

In [20]:
def ridge(df, inc):
    poly_deg = 2

    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)

    reg = Ridge(solver="cholesky", random_state=42) 
    model = Pipeline([
        ("poly_features", PolynomialFeatures(include_bias=False)), 
        ("regul_reg", reg)  
    ]) 

    x = df
    y = inc['incidents_per_month']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0) 

    grid = {
        "poly_features__degree": [2, 3, 4, 5, 6, 7, 8, 9, 10], 
        "regul_reg__alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10] 
    }

    gridse = GridSearchCV(
        model, 
        grid, 
        cv=5,  
        scoring='neg_mean_squared_error', 
        verbose=2,
        n_jobs=-1 
    )
    gridse.fit(x_train, y_train)
    legj = gridse.best_estimator_
    pred = legj.predict(x_test)
    mse = mean_squared_error(y_test, pred)
    print(mse)
    return legj, mse

"""mm_df_ridge = ridge(mm_df, incm)
ss_df2_ridge = ridge(ss_df2, incm2)
ss_df_ridge = ridge(ss_df, incm)
robust_df2_ridge = ridge(robust_df2, incm2)
robust_df_ridge = ridge(robust_df, incm)
"""
mm_df_ridge = ridge(mm_df, mmi)
mm_df2_ridge = ridge(mm_df2, mmi2)
ss_df2_ridge = ridge(ss_df2, ssi2)
ss_df_ridge = ridge(ss_df, ssi)
robust_df2_ridge = ridge(robust_df2, ri2)
robust_df_ridge = ridge(robust_df, ri)


Fitting 5 folds for each of 63 candidates, totalling 315 fits


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
280 fits failed out of a total of 315.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

0.02079296249794513
Fitting 5 folds for each of 63 candidates, totalling 315 fits


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
280 fits failed out of a total of 315.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

0.01876132639600804
Fitting 5 folds for each of 63 candidates, totalling 315 fits


...

0.6077673567944533
Fitting 5 folds for each of 63 candidates, totalling 315 fits


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
280 fits failed out of a total of 315.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

7.442969543701547
Fitting 5 folds for each of 63 candidates, totalling 315 fits


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
280 fits failed out of a total of 315.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local

7.505368120109088


In [ ]:
#ne futtasd le, mert megöli
mm_df_ae_ridge = ridge(mm_df_ae, mmi)
mm_df2_ae_ridge = ridge(mm_df2_ae, mmi2)
ss_df2_ae_ridge = ridge(ss_df2_ae, ssi2)
ss_df_ae_ridge = ridge(ss_df_ae, ssi)
robust_df2_ae_ridge = ridge(robust_df2_ae, ri2)
robust_df_ae_ridge = ridge(robust_df_ae, ri)


Fitting 5 folds for each of 63 candidates, totalling 315 fits


KeyboardInterrupt: 

In [ ]:
def ridge1(df, inc, ps, rs):

    legjemese=0
    for p in ps:
        for r in rs:
            if 'Unnamed: 0' in df.columns:
                df.index=df['Unnamed: 0']
                df.drop(columns= "Unnamed: 0", inplace=True)
            
            poly_deg = p
            ridge_alpha = r

            reg = Ridge(alpha=ridge_alpha, solver="cholesky", random_state=42) 
            model = Pipeline([
                ("poly_features", PolynomialFeatures(degree=poly_deg, include_bias=False)), 
                ("regul_reg", reg)
            ]) 
            x=df
            y=inc['incidents_per_month']
            x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.3, random_state = 0) 

            model.fit(x_train, y_train)
            pred = model.predict(x_test)
            mse = mean_squared_error(y_test, pred)
            if legjemese==0 or mse<legjemese:
                legjemese=mse
    print(legjemese)
    return mse
ps=[2, 3, 4] #ez lehetne több is (the same goes for rs), de már így is funky az általánosítóképessége
rs=[0.01, 0.1, 1]



In [ ]:
mm_df_ae_ridge = ridge1(mm_df_ae, incm, ps, rs)
ss_df2_ae_ridge = ridge1(ss_df2_ae, incm2, ps, rs)
ss_df_ae_ridge = ridge1(ss_df_ae, incm, ps, rs)
#scaled y-nal
mm_df_ae_ridge = ridge1(mm_df_ae, mmi, ps, rs)
ss_df2_ae_ridge = ridge1(ss_df2_ae, ssi2, ps, rs)
ss_df_ae_ridge = ridge1(ss_df_ae, ssi, ps, rs)
mm_df2_ae_ridge = ridge1(mm_df2_ae, mmi2, ps, rs)
#robust_df2_ae_ridge = ridge1(robust_df2_ae, ri2, ps, rs)
#robust_df_ae_ridge = ridge1(robust_df2_ae, ri, ps, rs)

mm_df_pca_ridge = ridge1(mm_df_pca, mmi, ps, rs)
ss_df2_pca_ridge = ridge1(ss_df2_pca, ssi2, ps, rs)
ss_df_pca_ridge = ridge1(ss_df_pca, ssi, ps, rs)
mm_df2_pca_ridge = ridge1(mm_df2_pca, mmi2, ps, rs)
#robust_df2_pca_ridge = ridge1(robust_df2_pca, ri2, ps, rs)
#robust_df_pca_ridge = ridge1(robust_df2_pca, ri, ps, rs)


5.147346970507769
6.831121004543727
6.62395178744461
0.011671988595259721
0.6666729726748558
0.6365317363905765
0.009375461531058832
0.027117379166615704
1.3718029361480182
1.4497824045856127
0.00913984828520432


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:237: LinAlgWarning: Ill-conditioned matrix (rcond=3.54772e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


In [ ]:
robust_df2_pca_ridge = ridge1(robust_df2_pca, ri2, ps, rs)
robust_df_pca_ridge = ridge1(robust_df_pca, ri, ps, rs)



0.37280414971102827
1.3304225267031007


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.60287e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.17628e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.46472e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Andris\

In [ ]:
ps1=[2]
rs1=[0.01, 0.1, 1]

robust_df2_ae_ridge = ridge1(robust_df2_ae, ri2, ps1, rs1)
robust_df_ae_ridge = ridge1(robust_df_ae, ri, ps1, rs1)



C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


6.599645238027443


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:237: LinAlgWarning: Ill-conditioned matrix (rcond=4.16114e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


2.587963814890297


In [ ]:
mm_df_pca_ridge = ridge1(mm_df_pca, incm, ps, rs)
mm_df2_pca_ridge = ridge1(mm_df2_pca, incm2, ps, rs) 
ss_df2_pca_ridge = ridge1(ss_df2_pca, incm2, ps, rs)
ss_df_pca_ridge = ridge1(ss_df_pca, incm, ps, rs)
robust_df2_pca_ridge = ridge1(robust_df2_pca, incm2, ps, rs)
robust_df_pca_ridge = ridge1(robust_df_pca, incm, ps, rs)

11.9587642124784
4.03067309377492
14.056294818156388
15.086897009590295
5.2425583555792015
11.973802740327898


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:237: LinAlgWarning: Ill-conditioned matrix (rcond=3.54772e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=6.60287e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.17628e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Andr

In [ ]:
#ridge+rfe
def ridgerse(df, inc, ps, rs):

    legjemese="b"
    for p in ps:
        for r in rs:

            
            poly_deg = p
            ridge_alpha = r

            reg = Ridge(alpha=ridge_alpha, solver="cholesky", random_state=42) 
            rfe = RFE(estimator=reg, n_features_to_select=int(len(df)/10))  
            model = Pipeline([
                ("poly_features", PolynomialFeatures(degree=poly_deg, include_bias=False)), 
                ("feature_selection", rfe),
                ("regul_reg", reg)
            ]) 
            x=df
            y=inc['incidents_per_month']
            print("x")
            x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.2, random_state = 0) 
            print("y")
            model.fit(x_train, y_train)
            pred = model.predict(x_test)
            mse = mean_squared_error(y_test, pred)
            print(mse)
            if legjemese=="b" or mse<legjemese:
                legjemese=mse

            
    #print(legjemese)
    return legjemese




In [ ]:
#ss_df_rrse=ridgerse(ss_df_ae, incm, ps, rs)
def ridge1earlyst(df, inc, ps, rs):

    legjlegjemese=float('inf')
    for p in ps:
        for r in rs:
            if 'Unnamed: 0' in df.columns:
                df.index=df['Unnamed: 0']
                df.drop(columns= "Unnamed: 0", inplace=True)
            
            poly_deg = p
            ridge_alpha = r

            reg = Ridge(alpha=ridge_alpha, solver="saga", max_iter=1, random_state=42) 
            model = Pipeline([
                ("poly_features", PolynomialFeatures(degree=poly_deg, include_bias=False)), 
                ("regul_reg", reg)
            ]) 
            x=df
            y=inc['incidents_per_month']
            x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.3, random_state = 0) 

            legjemese = float('inf')
            patience=10
            noimp = 0

            for epoch in range(100):
                
                model.fit(x_train, y_train)

          
                pred = model.predict(x_test)
                mse = mean_squared_error(y_test, pred)


                if mse < legjemese:
                    legjemese = mse
                    noimp = 0
                else:
                    noimp += 1

                if noimp >= patience:
            
                    break

            if legjemese < legjlegjemese:
                legjlegjemese = legjemese
                bestepoch=epoch
                bestmodel=clone(model)
    print(legjlegjemese)
    return legjlegjemese, bestepoch, bestmodel

#todo: ridgerfe+early stopping, lasso, sima poly, poly+reg, early stopping a többihez is, grad boosting+early stopping

In [ ]:

ss_df_ae_ridge_es=ridge1earlyst(ss_df_ae, ssi, ps, rs)
ss_df2_ae_ridge_es=ridge1earlyst(ss_df2_ae, ssi2, ps, rs)
mm_df_df_ae_ridge_es=ridge1earlyst(mm_df_ae, mmi, ps, rs)
mm_df2_ae_ridge_es=ridge1earlyst(mm_df2_ae, mmi2, ps, rs)
#robust_df_ae_ridge_es=ridge1earlyst(robust_df_ae, ri, ps, rs)
#robust_df2_ae_ridge_es=ridge1earlyst(robust_df2_ae, ri2, ps, rs)

ss_df_pca_ridge_es=ridge1earlyst(ss_df_pca, ssi, ps, rs)
ss_df2_pca_ridge_es=ridge1earlyst(ss_df2_pca, ssi2, ps, rs)
mm_df_df_pca_ridge_es=ridge1earlyst(mm_df_pca, mmi, ps, rs)
mm_df2_pca_ridge_es=ridge1earlyst(mm_df2_pca, mmi2, ps, rs)
#robust_df_pca_ridge_es=ridge1earlyst(robust_df_pca, ri, ps, rs)
#robust_df2_pca_ridge_es=ridge1earlyst(robust_df2_pca, ri2, ps, rs)


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: C

In [ ]:
robust_df_pca_ridge_es=ridge1earlyst(robust_df_pca, ri, ps, rs)
robust_df2_pca_ridge_es=ridge1earlyst(robust_df2_pca, ri2, ps, rs)

C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: C

In [ ]:
#lasso
def indiana(df, inc, ps, rs):
    legjemese=0
    for p in ps:
        for r in rs:
            if 'Unnamed: 0' in df.columns:
                df.index=df['Unnamed: 0']
                df.drop(columns= "Unnamed: 0", inplace=True)
            
            poly_deg = p
            lasso_alpha = r

            reg = Lasso(alpha=lasso_alpha)
            model = Pipeline([
                ("poly_features", PolynomialFeatures(degree=poly_deg, include_bias=False)), 
                ("regul_reg", reg)
            ]) 
            x=df
            y=inc['incidents_per_month']
            x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.3, random_state = 0) 

            model.fit(x_train, y_train)
            pred = model.predict(x_test)
            mse = mean_squared_error(y_test, pred)
            if legjemese==0 or mse<legjemese:
                legjemese=mse
    print(legjemese)
    return mse





In [ ]:
ss_df_ae_lasso=indiana(ss_df_ae, ssi, ps, rs)
ss_df2_ae_lasso=indiana(ss_df2_ae, ssi2, ps, rs)
mm_df_df_ae_lasso=indiana(mm_df_ae, mmi, ps, rs)
mm_df2_ae_lasso=indiana(mm_df2_ae, mmi2, ps, rs)
#robust_df_ae_lasso=indiana(robust_df_ae, ri, ps, rs)
#robust_df2_ae_lasso=indiana(robust_df2_ae, ri2, ps, rs)

ss_df_pca_ridge_lasso=indiana(ss_df_pca, ssi, ps, rs)
ss_df2_pca_ridge_lasso=indiana(ss_df2_pca, ssi2, ps, rs)
mm_df_df_pca_ridge_lasso=indiana(mm_df_pca, mmi, ps, rs)
mm_df2_pca_ridge_lasso=indiana(mm_df2_pca, mmi2, ps, rs)
#robust_df_pca_lasso=indiana(robust_df_pca, ri, ps, rs)
#robust_df2_pca_lasso=indiana(robust_df2_pca, ri2, ps, rs)

C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.899e+00, tolerance: 8.825e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.298e-01, tolerance: 8.825e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\l

0.5394084397437958


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.313e+00, tolerance: 1.144e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.176e+00, tolerance: 1.144e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\l

0.3650491880745443


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.724e-02, tolerance: 2.082e-04
  model = cd_fast.enet_coordinate_descent(


...

C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.564e-04, tolerance: 2.082e-04
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.787e-01, tolerance: 2.082e-04
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\l

0.013063366042097914


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.295e-02, tolerance: 2.659e-04
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.818e-01, tolerance: 2.659e-04
  model = cd_fast.enet_coordinate_descent(


0.008310443261801864


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.969e-02, tolerance: 2.659e-04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
robust_df_pca_lasso=indiana(robust_df_pca, ri, ps, rs)
robust_df2_pca_lasso=indiana(robust_df2_pca, ri2, ps, rs)

1.2789145928470738
0.4398904692191416


C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.560e+01, tolerance: 1.020e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.478e+01, tolerance: 1.020e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Andris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\l

In [24]:
#lasso+es
def lassoes(df, inc, ps, rs):

    legjlegjemese=float('inf')
    for p in ps:
        for r in rs:
            if 'Unnamed: 0' in df.columns:
                df.index=df['Unnamed: 0']
                df.drop(columns= "Unnamed: 0", inplace=True)
            
       

            reg = Lasso(alpha=r, warm_start=True, max_iter=1, random_state=42) 
            model = Pipeline([
                ("poly_features", PolynomialFeatures(degree=p, include_bias=False)), 
                ("regul_reg", reg)
            ]) 
            x=df
            y=inc['incidents_per_month']
            x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.3, random_state = 0) 

            legjemese = float('inf')
            patience=10
            noimp = 0

            for epoch in range(1000):
                
                model.fit(x_train, y_train)

          
                pred = model.predict(x_test)
                mse = mean_squared_error(y_test, pred)


                if mse < legjemese:
                    legjemese = mse
                    noimp = 0
                else:
                    noimp += 1

                if noimp >= patience:
            
                    break

            if legjemese < legjlegjemese:
                legjlegjemese = legjemese
                bestepoch=epoch
                bestmodel=clone(model)
    print(legjlegjemese)
    return legjlegjemese, bestepoch, bestmodel

In [25]:

ss_df_ae_lasso_es=lassoes(ss_df_ae, ssi, ps, rs)
ss_df2_ae_lasso_es=lassoes(ss_df2_ae, ssi2, ps, rs)
mm_df_df_ae_lasso_es=lassoes(mm_df_ae, mmi, ps, rs)
mm_df2_ae_ridge_es=lassoes(mm_df2_ae, mmi2, ps, rs)
#robust_df_ae_ridge_es=ridge1earlyst(robust_df_ae, ri, ps, rs)
#robust_df2_ae_ridge_es=ridge1earlyst(robust_df2_ae, ri2, ps, rs)

ss_df_pca_lasso_es=lassoes(ss_df_pca, ssi, ps, rs)
ss_df2_pca_lasso_es=lassoes(ss_df2_pca, ssi2, ps, rs)
mm_df_df_pca_lasso_es=lassoes(mm_df_pca, mmi, ps, rs)
mm_df2_pca_lasso_es=lassoes(mm_df2_pca, mmi2, ps, rs)
#robust_df_pca_lasso_es=lassoes(robust_df_pca, ri, ps, rs)
#robust_df2_pca_lasso_es=lassoes(robust_df2_pca, ri2, ps, rs)

NameError: name 'ps' is not defined

In [22]:
#sima linreg+es
#lasso+es
def eslinreg(df, inc):

    y=inc['incidents_per_month']
    df=df
    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)
    x=df
    x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.2, random_state = 0) 

    legjemese=float('inf')
    patience=10
    noimp = 0

    for epoch in range(100):
        lm = LinearRegression() 
        lm.fit(x_train, y_train) 
        pred = lm.predict(x_test) 
        mse = mean_squared_error(y_test, pred)
        if mse < legjemese:
            legjemese = mse
            noimp = 0
        else:
            noimp += 1

        if noimp >= patience:
            bestepoch=epoch
            bestmodel=clone(lm)
            break

    print(legjemese)

    #print(y_test)
    
    return legjemese, bestepoch, bestmodel

In [23]:
#linreg_es
ss_df_ae_linreg_es=eslinreg(ss_df_ae, ssi)
ss_df2_ae_linreg_es=eslinreg(ss_df2_ae, ssi2)
mm_df_df_ae_linreg_es=eslinreg(mm_df_ae, mmi)
mm_df2_ae_ridge_es=eslinreg(mm_df2_ae, mmi2)
robust_df_ae_ridge_es=eslinreg(robust_df_ae, ri)
robust_df2_ae_ridge_es=eslinreg(robust_df2_ae, ri2)

ss_df_pca_linreg_es=eslinreg(ss_df_pca, ssi)
ss_df2_pca_linreg_es=eslinreg(ss_df2_pca, ssi2)
mm_df_df_pca_linreg_es=eslinreg(mm_df_pca, mmi)
mm_df2_pca_linreg_es=eslinreg(mm_df2_pca, mmi2)
robust_df_pca_linreg_es=eslinreg(robust_df_pca, ri)
robust_df2_pca_linreg_es=eslinreg(robust_df2_pca, ri2)



0.6785148030277635
0.5198062169362567
0.022010282521405154
0.013574889687519173
1.846634316797844
3.6724437347477763
0.6450028456353498
0.49626021318616553
0.014586382422822843
0.010267571457947648
1.8622996065542006
0.5717059511398858


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor



#gradient descent
def graddescent(df, inc):


    y=inc['incidents_per_month']
    df=df
    if 'Unnamed: 0' in df.columns:
        df.index=df['Unnamed: 0']
        df.drop(columns= "Unnamed: 0", inplace=True)
    x=df
    x_train, x_test, y_train, y_test = train_test_split( 
            x, y, test_size = 0.2, random_state = 0) 
    
    model = GradientBoostingRegressor()
    model.fit(x_train,y_train)
    pred=model.predict(x_test)
    mse=mean_squared_error(y_test, pred)
    print(mse)
    return mse



ss_df_ae_gd=graddescent(ss_df_ae, ssi)
mm_df_ae_gd=graddescent(mm_df_ae, mmi)
robust_df_ae_gd=graddescent(robust_df_ae, ri)
ss_df2_ae_gd=graddescent(ss_df2_ae, ssi2)
mm_df2_ae_gd=graddescent(mm_df2_ae, mmi2)
robust_df2_ae_gd=graddescent(robust_df2_ae, ri2)

ss_df_pca_gd=graddescent(ss_df_pca, ssi)
mm_df_pca_gd=graddescent(mm_df_pca, mmi)
robust_df_pca_gd=graddescent(robust_df_pca, ri)
ss_df2_pca_gd=graddescent(ss_df2_pca, ssi2)
mm_df2_pca_gd=graddescent(mm_df2_pca, mmi2)
robust_df2_pca_gd=graddescent(robust_df2_pca, ri2)
#unpca-d

ss_df_gd=graddescent(ss_df, ssi)
mm_df_gd=graddescent(mm_df, mmi)
robust_df_gd=graddescent(robust_df, ri)
ss_df2_gd=graddescent(ss_df2, ssi2)
mm_df2_gd=graddescent(mm_df2, mmi2)
robust_df2_gd=graddescent(robust_df2, ri2)

0.7092791549826871
0.023905791422505894
0.8482772052168882
0.37515465597863595
0.008960693493803061
0.20172499725775822
0.7872837367204405
0.017200242680490186
0.8483310795112694
0.4012206626890497
0.010736394872669119
0.3040373174078115
0.7345874433104193
0.018767395483758675
0.8657958088157509
0.2784923187619994
0.0065526360965874175
0.20067781521026623


In [ ]:

"""

#ridge_es szűretlenen
ss_df_ridge_es=ridge1earlyst(ss_df, ssi, ps, rs)
ss_df2_ridge_es=ridge1earlyst(ss_df2, ssi2, ps, rs)
mm_df_ridge_es=ridge1earlyst(mm_df, mmi, ps, rs)
mm_df2_ridge_es=ridge1earlyst(mm_df2, mmi2, ps, rs)
#lasso_es szűretlenen
ss_df_lasso_es=lassoes(ss_df, ssi, ps, rs)
ss_df2_lasso_es=lassoes(ss_df2, ssi2, ps, rs)
mm_df_lasso_es=lassoes(mm_df, mmi, ps, rs)
mm_df2_lasso_es=lassoes(mm_df2, mmi2, ps, rs)
#robust_df_ridge_es=ridge1earlyst(robust_df, ri, ps, rs)
#robust_df2_ridge_es=ridge1earlyst(robust_df2, ri2, ps, rs)
"""
#ehhez sincs sajna elég memória



'\n\n#ridge_es szűretlenen\nss_df_ridge_es=ridge1earlyst(ss_df, ssi, ps, rs)\nss_df2_ridge_es=ridge1earlyst(ss_df2, ssi2, ps, rs)\nmm_df_ridge_es=ridge1earlyst(mm_df, mmi, ps, rs)\nmm_df2_ridge_es=ridge1earlyst(mm_df2, mmi2, ps, rs)\n#lasso_es szűretlenen\nss_df_lasso_es=lassoes(ss_df, ssi, ps, rs)\nss_df2_lasso_es=lassoes(ss_df2, ssi2, ps, rs)\nmm_df_lasso_es=lassoes(mm_df, mmi, ps, rs)\nmm_df2_lasso_es=lassoes(mm_df2, mmi2, ps, rs)\n#robust_df_ridge_es=ridge1earlyst(robust_df, ri, ps, rs)\n#robust_df2_ridge_es=ridge1earlyst(robust_df2, ri2, ps, rs)\n'

In [ ]:
"""#lasso szűretlenen
ss_df_lasso=indiana(ss_df, ssi, ps, rs)
ss_df2_lasso=indiana(ss_df2, ssi2, ps, rs)
mm_df_lasso=indiana(mm_df, mmi, ps, rs)
mm_df2_lasso=indiana(mm_df2, mmi2, ps, rs)

#ridge szűretlenen
ss_df_ridge=ridge1(ss_df, ssi, ps, rs)
ss_df2_ridge=ridge1(ss_df2, ssi2, ps, rs)
mm_df_ridge=ridge1(mm_df, mmi, ps, rs)
mm_df2_ridge=ridge1(mm_df2, mmi2, ps, rs)"""

#ez kimaxolja a memóriát, ne futtasd le